In [1]:
import numpy as np
import netCDF4 as nc

In [ ]:
# Location of data
out_dir = '/parent/directory/to/simulations'
sim_name = 'CODT_NAME_'
sims = ['ADB','S100','S95'] # list of simulation subnames

In [2]:
# Info for each particle injected into the domain
particle_var_names = ['PID', 'SID', 'DSR']
particle_var_dtype = [np.int32, np.int32, np.float64]
particle_var_long_names = ['Particle ID', 'Solute ID', 'Dry-Solute Radius']
particle_var_units = ['index #', 'index #', 'nm']
particle_dtype = np.dtype([('PID',np.int32),('SID',np.int32),('DSR',np.float64)])

# Info for each particle's trajectory
var_names = ['GRD', 'LOC', 'T', 'QV', 'SS', 'R', 'ACT', 'TIME']
var_long_names = ['Gridcell', 'Location', 'Temperature', 'Water Vapor', 'Supersaturation', 'Radius', 'Activation', 'Time']
var_units = ['index #', 'meters', 'celsius', 'g/kg', '%', 'microns', 'Boolean', 'Seconds']
var_dtype = np.dtype([('GRD',np.int32),('LOC',np.float64),('T',np.float64),('QV',np.float64),('SS',np.float64),('R',np.float64),('ACT',np.int8), ('TIME',np.float64)])

stream_var_dtype = np.dtype([('PID',np.int32),('data',var_dtype)])

In [3]:
#@njit
def create_ragged_array(PIDs, ID_data, VAR_data):
    '''
    PIDs - array of unique particle ID's (i.e. netcdf particle coordinate)
    ID_data - array of particle ID's from particle data bitstream
    VAR_data - array of specific variable's data corresponding to ID_data
    '''

    try:
        assert(ID_data.shape == VAR_data.shape)
    except:
        print('ID Data and Variable Data are not same shape')
        exit

    # Ragged array creation
    trajectories = np.empty_like(PIDs, dtype=object)
    for i, p in enumerate(PIDs):
        trajectories[i] = VAR_data[ID_data==p]

    return trajectories

In [ ]:
for s, sim in enumerate(sims):
    print(f'working on {sim}')

    # Parse data into numpy arrays of objects/voids
    profiles = nc.Dataset(f'{out_dir}/{sim_name}{sim}/{sim_name}{sim}.nc','r')
    particle_ids = np.fromfile(f'{out_dir}/{sim_name}{sim}/{sim_name}{sim}_PID.bin', dtype=particle_dtype)
    particle_data = np.fromfile(f'{out_dir}/{sim_name}{sim}/{sim_name}{sim}_particle.bin', dtype=stream_var_dtype)

    # Create the netcdf file
    nc_Path = f'{out_dir}/{sim_name}{sim}/{sim_name}{sim}_particles.nc'
    try: ncf.close()
    except: pass
    with nc.Dataset(nc_Path, mode='w', format='NETCDF4') as ncf:
        
        # Create particle dimension
        Np = particle_ids.shape[0] # Number of unique particles
        p_dim = ncf.createDimension('particle', Np)

        # Create/populate variables which describe properties of particle
        for i, name in enumerate(particle_var_names):
            var = ncf.createVariable(name, particle_var_dtype[i], 'particle')
            var.setncattr('Long Name', particle_var_long_names[i])
            var.setncattr('Units', particle_var_units[i])
            var[:] = particle_ids[name]

        # Create variable length array
        float_vl = ncf.createVLType(np.float64, 'float_vl')
        int_vl = ncf.createVLType(np.int32, 'int_vl')
        bool_vl = ncf.createVLType(np.int8, 'bool_vl')

        for i, name in enumerate(var_names):
            # Determine data type
            if var_dtype[i] == 'int32':
                vl = int_vl
            elif var_dtype[i] == 'float64':
                vl = float_vl
            elif var_dtype[i] == 'int8':
                vl = bool_vl
            else:
                print('ERROR: Unknown Variable Length Datatype')
            # Set up variable
            var = ncf.createVariable(name, vl, 'particle')
            # Set Attributes
            var.setncattr('Long Name', var_long_names[i])
            var.setncattr('Units', var_units[i])
            # Use the netcdf variable 'PID' to select the corresponding ragged array of each particle variable
            ragged_var_array = create_ragged_array(ncf['PID'], particle_data['PID'],particle_data['data'][name])
            var[:] = ragged_var_array # netcdf 'PID' order ensures placement in appropriate dimension


working on ADB
working on S100
working on S95
